# Description du cas

Cet exercice a pour but la manipulation de données structurées au format Parquet.

Nous allons traiter les données d'une entreprise industrielle fictive, Blabla Inc. Les données concernent 2 **sites** (usines) de l'entreprise : Naves et Villemomble.

Chaque site comporte plusieurs **ateliers** (_workshops_), et chaque atelier possède une ou plusieurs **machines**.

Enfin, chaque machine enregistre son état à une fréquence donnée, sous forme de **séries temporelles**.

## Ce qui est demandé

Il est demandé de lire les données d'entrée (JSON et CSV), de les traiter en mémoire, et de les mettre en forme avant de les stocker au **format Parquet** selon un modèle attendu.

Lorsque vous voyez un marqueur du type `### CHANGE ME ###` dans le code, ou dans un commentaire, c'est qu'il y a quelque chose à compléter pour accomplir une tâche.

Le notebook va vous guider dans cette tâche. Voyons d'abord les données à manipuler.

## Les données

Les données se trouvent dans le répertoire `data/` de votre notebook.

### L'organisation des sites et ateliers

L'organisation de Blabla Inc. est décrite de manière hiérarchique dans le fichier `equipments.json`, qui contient un objet `organization` avec la structure suivante :
- `name` : nom de l'entreprise
- `sites` : liste des sites de l'entreprise, avec pour chacun :
  - `name` : nom du site
  - `workshops` : liste des ateliers, avec pour chacun :
    - `name` : nom de l'atelier
    - `machines` : liste des ID de machines présentes dans l'atelier (de 1 à 3 machines selon le site et l'atelier)

<u>Extrait du fichier :</u>
```json
{
  "organization": {
    "name": "Blabla Inc.",
    "sites": [
      {
        "name": "Naves",
        "workshops": [
          {"name": "Préparation", "machines": ["NP1"]},
          {"name": "Usinage", "machines": ["NU1", "NU2", "NU3"]},
          ...
        ]
      },
      ...
    ]
  }
}
```

### Les "mappings" de noms de machines

Les ID de machines qui sont dans le fichier d'organisation, sont des noms de code. Dans l'exercice, il est demandé de mettre à la place les noms "lisibles" des machines ; pour cela il faut une correspondance.

La table de correspondance est dans le fichier `mapping.csv`, qui liste toutes les machines avec pour chacune :
- `machine_id` : ID codé de la machine (même nomenclature que dans `equipments.json`)
- `machine_name` : nom lisible de la machine (celui à reprendre dans les fichier de sortie)

<u>Extrait du fichier :</u>
```
machine_id;machine_name
NP1;Golgoth 3000
NU1;Fraisator-1
NU2;Fraisator-2
NU3;Fraisator-3
NA1;Lego_A
...
```

### Les séries temporelles

Les séries sont des groupes de fichiers, dans le répertoire `data/ts/`, avec plusieurs fichiers par machine. Les fichiers sont nommés `ID_n.csv`, où `ID` est l'ID de machine, et `n` le numéro de séquence du fichier, de 1 à 26 : chaque série est découpée par blocs de 8h.

La structure de chaque fichier CSV est la suivante :
- `timestamp` : la date et l'heure du point de mesure (format aaaa-mm-jj hh:mm:ss)
- un champ par type de mesure, en fonction de l'atelier où se trouve la machine :
  - "Préparation" : `scanned_products` est un entier donnant le nombre de produits pointés à l'étape de préparation
  - "Usinage" : `rotation_x`, `rotation_y` et `rotation_z` sont 3 flottants indiquant l'orientation de la tête d'usinage, en degrés
  - "Assemblage" : `products` est un entier donnant le nombre de produits assemblés
  - "Four" : `temperature` est la température du four (en °C), `humidity` son degré d'hygrométrie (en %)
  - "Métrologie" : `products` est un entier donnant le nombre de produits finis contrôlés, et `width`, `length` et `height` (en cm) le résultat de la dernière mesure

Noter que les timestamps sont irréguliers, et que les fréquences ne sont pas les mêmes d'un atelier à l'autre.

Tous les fichiers ont les noms de champs en en-tête.

<u>Exemple pour un four :</u>
```
timestamp,temperature,humidity
2024-01-28 00:00:07,812,93
2024-01-28 00:00:17,812,93
2024-01-28 00:00:27,812,93
...
```

# Imports de packages Python
Nous aurons besoin de ces packages pour manipuler les fichiers.

- `pandas` : manipulation de dataframes en mémoire
- `json` : pour la lecture des données JSON
- `glob` : énumération des fichiers de données dans un répertoire
- `os` : nous utiliserons ici les fonctions de manipulation de chemins de fichiers

In [1]:
import pandas as pd
import json
import glob
import os

# Lecture des données

## Organisation
C'est un fichier JSON que nous pouvons lire directement, ce qui donne un dictionnaire Python.

In [2]:
with open('data/equipments.json', 'r') as f:
    equipments = json.load(f)
    
equipments

{'organization': {'name': 'Blabla Inc.',
  'sites': [{'name': 'Naves',
    'workshops': [{'name': 'Préparation', 'machines': ['NP1']},
     {'name': 'Usinage', 'machines': ['NU1', 'NU2', 'NU3']},
     {'name': 'Assemblage', 'machines': ['NA1', 'NA2']}]},
   {'name': 'Villemomble',
    'workshops': [{'name': 'Préparation', 'machines': ['VP1', 'VP2']},
     {'name': 'Usinage', 'machines': ['VU1', 'VU2']},
     {'name': 'Four', 'machines': ['VF1']},
     {'name': 'Assemblage', 'machines': ['VA1', 'VA2']},
     {'name': 'Métrologie', 'machines': ['VM1']}]}]}}

## Mapping des noms de machines
C'est un fichier CSV, mais nous avons besoin de le transformer en dictionnaire donnant, pour chaque ID de machine, son nom. Nous l'enrichissons ensuite avec d'autres infos de l'équipement, qui indiquent où se trouve la machine (site, atelier).

Nous utilisons la fonction `read_csv()` de Pandas, qui permet de lire un fichier à partir de son nom / chemin, en précisant le caractère (`sep`) qui sépare les champs.

In [ ]:
# 1. Lecture du fichier sous forme de dataframe
#
# Attendu : un dataframe de la forme (ordre des lignes indifférent, et ne pas tenir compte de l'index de ligne (nombres en gras) qui s'afficheront en sortie) :
#
# +------------+--------------+
# | machine_id | machine_name |
# +------------+--------------+
# |        NP1 | Golgoth 3000 |
# |        NU1 |  Fraisator-1 |
# |        NU2 |  Fraisator-2 |
# |        ... |          ... |
# |        VM1 |     Sherlock |
# +------------+--------------+

mapping_df = pd.read_csv('### CHANGE ME ###', sep='### CHANGE ME ###')

mapping_df

In [ ]:
# 2. Transformation en dictionnaire Python, en itérant sur les lignes du dataframe
# On fait un dictionnaire de petits dictionnaires {'machine_name': xxx}, pour pouvoir l'enrichir dans l'étape d'après
#
# Attendu : un dictionnaire de la forme :
#
# {'NP1': {'machine_name': 'Golgoth 3000'},
#  'NU1': {'machine_name': 'Fraisator-1'},
#  'NU2': {'machine_name': 'Fraisator-2'},
#  'NU3': {'machine_name': 'Fraisator-3'},
#  'NA1': {'machine_name': 'Lego_A'},
#  ...
#  'VP2': {'machine_name': 'Cleanomatic-2'}]

mapping = {
    row['machine_id']: { ### CHANGE ME ### }
    for _, row in mapping_df.iterrows()
}

mapping

In [ ]:
# 3. Enrichissement avec les données d'équipement
# A l'issue de cette étape, chaque ID de machine est associé à toutes les infos de la machine
#
# Attendu : un dictionnaire de la forme :
#
# {'NP1': {'machine_name': 'Golgoth 3000', 'site': 'Naves', 'workshop': 'Préparation'},
#  'NU1': {'machine_name': 'Fraisator-1', 'site': 'Naves', 'workshop': 'Usinage'},
#  'NU2': {'machine_name': 'Fraisator-2', 'site': 'Naves', 'workshop': 'Usinage'},
#  'NU3': {'machine_name': 'Fraisator-3', 'site': 'Naves', 'workshop': 'Usinage'},
#  'NA1': {'machine_name': 'Lego_A', 'site': 'Naves', 'workshop': 'Assemblage'},
#  ...
#  'VM1': {'machine_name': 'Sherlock', 'site': 'Villemomble', 'workshop': 'Métrologie'}}
 
for site in equipments['### CHANGE ME ###']['### CHANGE ME ###']:
    for workshop in site['### CHANGE ME ###']:
        for machine_id in workshop['### CHANGE ME ###']:
            mapping_machine = mapping[machine_id]
            ### CHANGE ME ### (ajouter le nom du site, le nom du workshop)

mapping

## Séries temporelles

Les séries sont dans plusieurs fichiers CSV, qu'on va regrouper sous forme d'un unique dataframe par machine (repérée par son ID, présent dans le nom de chaque fichier).

In [ ]:
# 1. Enumération des fichiers de séries, dans le répertoire correspondant
ts_filenames = glob.glob('data/ts/*.csv')
print(f'{len(ts_filenames)} fichiers de séries')

Ceci vous sera utile pour écrire le bout de code manquant :
- un paramètre supplémentaire à la fonction `read_csv()` : `parse_dates`
  - quand le fichier contient des dates, il faut aider Pandas à les détecter en indiquant les noms des champs concernés
  - concrètement, il vous faudra ajouter un paramètre `parse_dates=['nom de champ']` pour lui dire quoi interpréter
- la fonction `concat()` de Pandas (`pd.concat()`) qui permet de mettre bout à bout 2 dataframes pour en créer un unique. On lui passe en paramètre une liste de dataframes à concaténer, ex. `[df1, df2, df3, ...]`

In [ ]:
# 2. Agrégation des séries. Pour chaque fichier CSV de série temporelle :
# - extraction de l'ID de machine, à partir du nom du fichier
# - lecture du fichier CSV dans un petit dataframe
# - concaténation avec le dataframe précédent de la machine
# Le tout est stocké dans un dictionnaire de dataframes, `dataframes_per_machine`
#
# Type de sortie attendue à la fin (ordre indifférent) :
#
# Machine NP1: 25397 échantillons
# Machine NU1: 166442 échantillons
# Machine NU2: 166357 échantillons
# Machine NU3: 166364 échantillons
# ...
# Machine VM1: 9423 échantillons
#
# +---------------------+-------------+------------+------------+
# |           timestamp | rotationx_x | rotation_y | rotation_z |
# +---------------------+-------------+------------+------------+
# | 2024-01-29 16:00:03 |   88.763247 | -49.428614 | 133.959195 |
# | 2024-01-29 16:00:08 |   90.583300 | -49.193791 | 134.367569 |
# | 2024-01-29 16:00:13 |   88.727491 | -52.121831 | 136.238264 |
# |                 ... |         ... |        ... |        ... |
# | 2024-01-26 15:59:56 | -174.516082 | 52.336152  |  79.645205 |
# +---------------------+-------------+------------+------------+


# Initialise le dictionnaire avec des dataframes vides
dataframes_per_machine = {
    machine_id: pd.DataFrame()
    for machine_id in mapping
}


# Parcourt les fichiers
for filename in ts_filenames:
    # Extraction de l'ID de machine
    machine_id = os.path.basename(filename).split('_')[0]
          
    small_dataframe = ### CHANGE ME ###
    dataframes_per_machine[machine_id] = ### CHANGE ME ###

for machine_id, machine_dataframe in dataframes_per_machine.items():
    print(f'Machine {machine_id}: {len(machine_dataframe)} échantillons')
    
dataframes_per_machine['NU2']

Pour ajouter une nouvelle colonne à un dataframe `df`, on peut utiliser la syntaxe suivante : `df['nouvelle_colonne'] = valeur`.

In [ ]:
# 3. Ajout au dataframe des infos de nom de machine, de site et d'atelier
#
# Attendu un dataframe enrichi :
# +---------------------+-------------+------------+------------+--------------+-------+----------+
# |           timestamp | rotationx_x | rotation_y | rotation_z | machine_name |  site | workshop |
# +---------------------+-------------+------------+------------+--------------+-------+----------+
# | 2024-01-29 16:00:03 |   88.763247 | -49.428614 | 133.959195 |  Fraisator-2 | Naves | Usinage  |
# | 2024-01-29 16:00:08 |   90.583300 | -49.193791 | 134.367569 |  Fraisator-2 | Naves | Usinage  |
# | 2024-01-29 16:00:13 |   88.727491 | -52.121831 | 136.238264 |  Fraisator-2 | Naves | Usinage  |
# |                 ... |         ... |        ... |        ... |  Fraisator-2 | Naves | Usinage  |
# | 2024-01-26 15:59:56 | -174.516082 | 52.336152  |  79.645205 |  Fraisator-2 | Naves | Usinage  |
# +---------------------+-------------+------------+------------+--------------+-------+----------+

for machine_id, df in dataframes_per_machine.items():
    ### CHANGE ME ###
    
dataframes_per_machine['NU2']

# Ecriture du résultat brut
On sauvegarde chaque dataframe de machine dans le répertoire de sortie, en itérant sur le dictionnaire créé précédemment.

Pandas a des fonctions `read_xxx()` (ex. `read_csv()`) pour la lecture, et à l'inverse, des fonctions `to_xxx()` (ex. `to_parquet()`) pour les écritures de dataframes en mémoire.

Le paramètre `index=False` empêche l'écriture des index de lignes (nombres en gras) dans le fichier, car dans notre cas ils n'ont pas de signification.

In [ ]:
for machine_id, machine_dataframe in dataframes_per_machine.items():
    machine_dataframe.to_parquet(f'data/ts_parquet/{machine_id}.parquet', index=False)

Vous pouvez consulter le répertoire `data/ts_parquet` dans le bandeau de gauche, pour constater que les fichiers ont bien été créés (un par machine).

# Ecriture du résultat - avec partitionnement

Ci-dessous, on extrait le jour du timestamp :
- la fonction `.dt.strftime(format)` transforme un champ date de Pandas en chaîne de caractères, en appliquant un format de représentation
- on ajoute une colonne virtuelle au dataframe `machine_dataframe` via la méthode `.assign(day=day)`. Contrairement à la notation `df['nouvelle_colonne'] = valeur`, le dataframe n'est pas modifié, mais les méthodes qu'on applique sur le résultat -- et uniquement celles-ci -- tiennent compte de cette colonne virtuelle
- noter que `day` n'est pas une valeur constante, mais une série, c'est-à-dire une liste de valeurs analogue à une colonne de dataframe (ce qui fait qu'on peut la lui assigner)

Puis on écrit le fichier partitionné comme il faut.

In [ ]:
for machine_id, machine_dataframe in dataframes_per_machine.items():
    day = machine_dataframe.timestamp.dt.strftime('%Y-%m-%d')
    machine_dataframe.assign(day=day).to_parquet(f'data/ts_parquet_part/{machine_id}', index=False, partition_cols=[### CHANGE ME ###])

Vous pouvez aller consulter le répertoire `data/ts_parquet_part` pour voir comment Pandas (et son partenaire PyArrow) stockent les fichiers Parquet partitionnés.

NB : si vous essayez plusieurs fois l'écriture partitionnée, les fichiers vont s'accumuler ; il faudrait normalement les supprimer avant mais nous ne le faisons pas ici, par simplicité.

# Un partitionnement plus fin

Sauvegarder une nouvelle version des données, avec un partitionnement à plusieurs niveaux :
- `year` (exemple : 2024)
  - `month` (exemple : 01 ou 1)
    - `day_in_month` (exemple : 29)

Ecrire ces données dans un répertoire différent (ex. `data/ts_parquet_part_ymd`) pour ne pas mélanger avec les fichiers de la question précédente.

# Partitionnement global

Si vous avez le temps, concaténez tous les dataframes dans `dataframes_per_machine` en un seul gros dataframe, et sauvez-le avec le partitionnement suivant :
- `site` (exemple : Naves)
  - `machine_name` (exemple : Lego_A)
    - `year` (exemple : 2024)
      - `month` (exemple : 01 ou 1)
        - `day_in_month` (exemple : 29)

Comme précédemment, écrivez-le dans un nouveau répertoire.

On peut vérifier que les données relues contiennent l'ensemble des partitions :

In [ ]:
pd.read_parquet('### CHANGE ME ###')

# Influence de la compression

Parquet est un format compressé, pour occuper moins de place et accélérer les lectures et écritures. On peut choisir plusieurs algorithmes de compression.

Consulter la [documentation de Pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_parquet.html#pandas.DataFrame.to_parquet), trouver le paramètre de l'algorithme de compression, et en tester quelques-uns en vérifiant la taille du fichier à chaque fois.

Tester avec un seul dataframe (par exemple la machine `NA1`), sans partitionnement.